# Transportation problem in PuLP

In [ ]:
from pulp import *

# Problem definition
model = LpProblem("Shipping_Cost_Minimization", LpMinimize)

suppliers = ["Albany", "Boston", "Hartford"]
hubs = ["Richmond", "Charlotte"]
warehouses = ["Dallas", "Louisville", "Memphis", "Nashville"]

# Decision variables
indices = [(i, j) for i in suppliers for j in hubs] + [(i, j) for i in hubs for j in warehouses]
variables = LpVariable.dicts("v", indices, lowBound=0, cat=LpContinuous)


In [ ]:
#create decision variable with V prefix of the list

cost = {
    ("Albany", "Richmond"): 55, ("Boston", "Richmond"): 46, ("Hartford", "Richmond"): 50,
    ("Albany", "Charlotte"): 40, ("Boston", "Charlotte"): 43, ("Hartford", "Charlotte"): 50,
    ("Charlotte", "Dallas"): 38, ("Charlotte", "Louisville"): 40, ("Charlotte", "Memphis"): 51, ("Charlotte", "Nashville"):40,
    ("Richmond", "Dallas"): 30, ("Richmond", "Louisville"): 47, ("Richmond", "Memphis"): 41, ("Richmond", "Nashville"):45
}



In [ ]:
# Objective function
model += lpSum(variables[ij] * cost[ij] for ij in variables), "Total shipping cost"

# Constraints
factories = ["Albany", "Boston", "Hartford"]
supplies = [450, 500, 580]
factories_supply = dict(zip(factories, supplies))

warehouses = ["Dallas", "Louisville", "Memphis", "Nashville"]
demands = [450, 300, 275, 400]
warehouses_demand = dict(zip(warehouses, demands))

In [ ]:
# Factory supply constraints
for factory, supply in factories_supply.items():
    model += lpSum(variables[factory, hub] for hub in hubs) <= supply, f"{factory} supply"

# Hub constraints
for hub in hubs:
    model += lpSum(variables[factory, hub] for factory in suppliers) - lpSum(variables[hub, warehouse] for warehouse in warehouses) == 0, f"{hub} storage"

# Warehouse demand constraints
for warehouse, demand in warehouses_demand.items():
    model += lpSum(variables[hub, warehouse] for hub in hubs) == demand, f"{warehouse} demand"


In [ ]:
# Solve the problem
model.solve()

# Print the results
print(f"Status: {LpStatus[model.status]}")

for variable in model.variables():
    print(f"{variable.name} = {variable.varValue}")

print(f"Objective function value: {value(model.objective)}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/serena/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/74/vt9gsry94p3b7s67kxd55vvm0000gn/T/8d2a9a5e347d49bcbd54deb597a618c0-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/74/vt9gsry94p3b7s67kxd55vvm0000gn/T/8d2a9a5e347d49bcbd54deb597a618c0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 57 RHS
At line 67 BOUNDS
At line 68 ENDATA
Problem MODEL has 9 rows, 14 columns and 28 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (-4) rows, 10 (-4) columns and 20 (-8) elements
0  Obj 65875 Primal inf 1425 (1) Dual inf 23.999997 (3)
6  Obj 119625
Optimal - objective value 119625
After Postsolve, objective 119625, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 119625 - 6 iterations time 0.002, Presolve 0.00
Option for 

In [ ]:
model

Shipping_Cost_Minimization:
MINIMIZE
40*v_('Albany',_'Charlotte') + 55*v_('Albany',_'Richmond') + 43*v_('Boston',_'Charlotte') + 46*v_('Boston',_'Richmond') + 38*v_('Charlotte',_'Dallas') + 40*v_('Charlotte',_'Louisville') + 51*v_('Charlotte',_'Memphis') + 40*v_('Charlotte',_'Nashville') + 50*v_('Hartford',_'Charlotte') + 50*v_('Hartford',_'Richmond') + 50*v_('Richmond',_'Dallas') + 47*v_('Richmond',_'Louisville') + 41*v_('Richmond',_'Memphis') + 45*v_('Richmond',_'Nashville') + 0
SUBJECT TO
Albany_supply: v_('Albany',_'Charlotte') + v_('Albany',_'Richmond') <= 450

Boston_supply: v_('Boston',_'Charlotte') + v_('Boston',_'Richmond') <= 500

Hartford_supply: v_('Hartford',_'Charlotte') + v_('Hartford',_'Richmond')
 <= 580

Richmond_storage: v_('Albany',_'Richmond') + v_('Boston',_'Richmond')
 + v_('Hartford',_'Richmond') - v_('Richmond',_'Dallas')
 - v_('Richmond',_'Louisville') - v_('Richmond',_'Memphis')
 - v_('Richmond',_'Nashville') = 0

Charlotte_storage: v_('Albany',_'Charlotte') 

## Calculating the reduced costs

In the context of the transportation problem, a non-basic variable is a decision variable that is currently not being used to ship items (i.e., its value is zero in the optimal solution). If the reduced cost for a non-basic variable is positive, it means that the variable's cost should decrease by that amount for it to become part of the optimal solution. In other words, if the cost associated with that particular shipping route decreases by the reduced cost value, it will become economical to use that route, and it will be included in the optimal solution.

In [ ]:
# Print the reduced costs for each decision variable
for variable in model.variables():
    print(f"Reduced cost for {variable.name}: {variable.dj}")

#how much cost must decrease for the decision variable to be used

'''
Intepretation:
Route Albany to Charlotte, Boston to Charlotte, Charlotte to Dallas, Charlotte to Louisville, Charlotte to Nashville,
Hartford to Charlotte, Hartford to Richmond, and Richmond to Memphis are already in use.

If other possible routes want to be included in the optimal solution, the cost of Albany to Richmond must reduce 15,
the cost of Boston to Richmond must reduce by 3, the cost of Charlotte to Memphis must reduce 10, the cost of Richmond
to Dallas must reduce by 12, the cost of Richmond to Louisville must reduce 7, and the cost of Richmond to Nashville
must reduce by 5.

If the company wish to expand route usage, they may want to start with route Boston to Richmond first because lowering
cost for this route is the cheapest among all other routes.
'''

Reduced cost for v_('Albany',_'Charlotte'): 0.0
Reduced cost for v_('Albany',_'Richmond'): 15.0
Reduced cost for v_('Boston',_'Charlotte'): 0.0
Reduced cost for v_('Boston',_'Richmond'): 3.0
Reduced cost for v_('Charlotte',_'Dallas'): 0.0
Reduced cost for v_('Charlotte',_'Louisville'): 0.0
Reduced cost for v_('Charlotte',_'Memphis'): 10.0
Reduced cost for v_('Charlotte',_'Nashville'): 0.0
Reduced cost for v_('Hartford',_'Charlotte'): 0.0
Reduced cost for v_('Hartford',_'Richmond'): 0.0
Reduced cost for v_('Richmond',_'Dallas'): 12.0
Reduced cost for v_('Richmond',_'Louisville'): 7.0
Reduced cost for v_('Richmond',_'Memphis'): 0.0
Reduced cost for v_('Richmond',_'Nashville'): 5.0


'\nIntepretation: \nRoute Albany to Charlotte, Boston to Charlotte, Charlotte to Dallas, Charlotte to Louisville, Charlotte to Nashville,\nHartford to Charlotte, Hartford to Richmond, and Richmond to Memphis are already in use.\n\nIf other possible routes want to be included in the optimal solution, the cost of Albany to Richmond must reduce 15, \nthe cost of Boston to Richmond must reduce by 3, the cost of Charlotte to Memphis must reduce 10, the cost of Richmond\nto Dallas must reduce by 12, the cost of Richmond to Louisville must reduce 7, and the cost of Richmond to Nashville\nmust reduce by 5.\n\nIf the company wish to expand route usage, they may want to start with route Boston to Richmond first because lowering\ncost for this route is the cheapest among all other routes.\n'

## Calculating the shadow price and slack

In [ ]:
import pandas as pd
# Print Shadow Price and Slack
o = [{'name': name, 'condition: ': str(c), 'constraint value':  model.constraints[name].value()-model.constraints[name].constant, 'shadow price': c.pi, 'slack': c.slack}
     for name, c in model.constraints.items()]
pd.DataFrame(o)

,name,condition:,constraint value,shadow price,slack
0,Albany_supply,"v_('Albany',_'Charlotte') + v_('Albany',_'Rich...",450.0,40.0,-0.0
1,Boston_supply,"v_('Boston',_'Charlotte') + v_('Boston',_'Rich...",500.0,43.0,-0.0
2,Hartford_supply,"v_('Hartford',_'Charlotte') + v_('Hartford',_'...",580.0,50.0,-0.0
3,Dallas_demand,"v_('Charlotte',_'Dallas') + v_('Richmond',_'Da...",450.0,38.0,-0.0
4,Louisville_demand,"v_('Charlotte',_'Louisville') + v_('Richmond',...",300.0,40.0,-0.0
5,Memphis_demand,"v_('Charlotte',_'Memphis') + v_('Richmond',_'M...",275.0,41.0,-0.0
6,Nashville_demand,"v_('Charlotte',_'Nashville') + v_('Richmond',_...",400.0,40.0,-0.0
7,Richmond_storage,"v_('Albany',_'Richmond') + v_('Boston',_'Richm...",0.0,0.0,-0.0
8,Charlotte_storage,"v_('Albany',_'Charlotte') + v_('Boston',_'Char...",105.0,0.0,-105.0
